In [4]:
import torch
import torch.optim as optim
import math

import torch.nn as nn

In [2]:
class FireflyOptimizer(optim.Optimizer):
    def __init__(self, params, func, dim, n_fireflies=20, alpha=0.5, beta0=1.0, gamma=1.0, lb=-10, ub=10, max_iter=100):
        defaults = dict(func=func, dim=dim, n_fireflies=n_fireflies, alpha=alpha, beta0=beta0, gamma=gamma, lb=lb, ub=ub, max_iter=max_iter)
        super(FireflyOptimizer, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            func = group['func']
            dim = group['dim']
            n_fireflies = group['n_fireflies']
            alpha = group['alpha']
            beta0 = group['beta0']
            gamma = group['gamma']
            lb = group['lb']
            ub = group['ub']
            max_iter = group['max_iter']

            # Initialize firefly positions
            fireflies = lb + (ub - lb) * torch.rand((n_fireflies, dim), device=self.param_groups[0]['params'][0].device)
            intensities = torch.zeros(n_fireflies, device=self.param_groups[0]['params'][0].device)

            # Main optimization loop
            for t in range(max_iter):
                # Evaluate intensities
                for i, param_group in enumerate(self.param_groups):
                    for j, p in enumerate(param_group['params']):
                        p.data += torch.rand_like(p.data) * 0.01  # Random update for demonstration purposes
                        intensities[i] = func(p.data)

                # Update firefly positions
                for i in range(n_fireflies):
                    for j in range(n_fireflies):
                        if intensities[j] < intensities[i]:
                            r = torch.norm(fireflies[i] - fireflies[j])
                            beta = beta0 * math.exp(-gamma * r**2)
                            fireflies[i] += beta * (fireflies[j] - fireflies[i]) + alpha * (torch.rand(dim) - 0.5)

                            # Ensure new position is within bounds
                            fireflies[i] = torch.clamp(fireflies[i], lb, ub)

            # Update parameter values
            for i, param_group in enumerate(self.param_groups):
                for j, p in enumerate(param_group['params']):
                    p.data = fireflies[i]

        return loss


In [16]:
# Define your model and loss function
model = nn.Linear(3, 1)
loss_func = nn.MSELoss()

In [17]:

# Create an instance of your custom FireflyOptimizer
optimizer = FireflyOptimizer(model.parameters(), lambda x: loss_func(model(inputs), targets), dim=3, max_iter=100)

In [18]:
inputs = torch.randn(100, 3)
targets = torch.mm(inputs, torch.Tensor([[1, 2, 3]]).t())

In [20]:
inputs = torch.randn(100, 3)
targets = torch.mm(inputs, torch.Tensor([[1, 2, 3]]).t())

# Reshape targets to be a column vector
targets = targets.view(-1, 1)

# Instantiate the FireflyAlgorithm optimizer
fa = FireflyAlgorithm(func, dim=3, max_iter=1000)

# Optimize the model
best_solution, best_fitness = fa.optimize()

# Set the best solution as the weights of the linear regression model
model.weight.data = torch.tensor(best_solution.reshape(1, -1))


NameError: name 'FireflyAlgorithm' is not defined

In [19]:

# Training loop
for _ in range(10):
    optimizer.zero_grad()
    loss = loss_func(model(inputs), targets)
    optimizer.step()

RuntimeError: mat2 must be a matrix, got 1-D tensor

In [12]:
model.weight

Parameter containing:
tensor([ 6.3520, -5.3422,  5.2594], requires_grad=True)